# Details

Use Genetic algorithm (GA) to find the best key. GA basically use the concept of natural selection. The process of natural selection starts with the selection of fittest individuals from a population. They produce offspring that will inherit the characteristic of their parents and will be added to the next generation. As a result, a generation with the fittest individual will be found. There are 5 phases in a GA:

1. initial population
2. fitness function
3. selection
4. crossover
5. mutation

## Part 1: English Validator

I use nltk package as the english validator. So the program will stop finding the best key when all of the words is identified as english

In [ ]:
import nltk
from nltk import FreqDist
import csv
import collections
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
!pip install wordsegment

In [ ]:
from wordsegment import load, isegment
load()

In [ ]:
from nltk.corpus import words, wordnet
from nltk.stem.wordnet import WordNetLemmatizer

setofwords = set(words.words())
lemma = WordNetLemmatizer()

def check(text):
    seg = isegment(text)
    count = 0
    for word in seg:
        w = lemma.lemmatize(word)
        if w not in setofwords and not wordnet.synsets(w):
            count +=1
            # print(word)
        
        if count == 2:
            return False

    return True

## Part 2: Genetic Algorithm (GA)

The fitness function I used is take the log difference between the correct ngram frequency and the current decrypted text (with current key). The ngram I used are one, bi and tri gram. Then sum the log difference of three of them as used the sum as the fitness score.

The correct frequency I used is from the Great Gatsby text. 


In [ ]:
import csv
import math
from random import randint, uniform

class GeneticSolver:

    # init class
    def __init__(self):
        self.gen = 500
        self.population_size = 500
        self.tournament_size = 20
        self.tournament_winner_proba = 0.75
        self.crossover_proba = 0.65
        self.crossover_points_count = 5
        self.mutation_proba = 0.4
        self.elitism_percentage = 0.15

        self.onegram_weight = 1.0
        self.bigram_weight = 1.0
        self.trigram_weight = 1.0

        self.letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        self.elitism_count = int(self.elitism_percentage * self.population_size)
        self.crossover_count = self.population_size - self.elitism_count

        self.tournament_proba = [self.tournament_winner_proba]

        for i in range(1, self.tournament_size):
            proba = self.tournament_proba[i-1] * (1.0 - self.tournament_winner_proba)
            self.tournament_proba.append(proba)
        
    # get current decrypt text ngrams    
    def generate_ngrams(self, word, n):
        ngrams = [word[i:i+n] for i in range(len(word)-n+1)]

        processed_ngrams = []
        for ngram in ngrams:
            if ngram.isalpha():
                ngram_upper = ngram.upper()
                processed_ngrams.append(ngram_upper)

        return processed_ngrams

    # get the freq of the great gatsby text
    def get_freq(self, text, n):
        words = text.split()
        ngrams = []
        ngram_frequency = {}
        for word in words:
            ngrams += [word[i:i+n] for i in range(len(word)-n+1)]

        processed_ngrams = []
        for ngram in ngrams:
            if ngram.isalpha():
                ngram_upper = ngram.upper()
                processed_ngrams.append(ngram_upper)

        frequency = FreqDist(processed_ngrams)

        sorted_frequency = sorted(frequency.items(), key=lambda x: x[1], reverse=True)
        frequency = collections.OrderedDict(sorted_frequency)

        for key, value in frequency.items():
            ngram_frequency[key] = value

        return ngram_frequency

    
    # decrypt text
    def decrypt(self, key):
        mapping = {}

        for i in range(26):
            k = self.letters[i]
            v = key.upper()[i]

            mapping[k] = v

        text = ''
        for ch in self.ciphertext:
            if ch not in self.letters:
                text += ch
            else:
                text += mapping[ch]
        
        return text

    # calculate fitness score function using log difference of one, bi, tri-grams
    def calc_fitness(self, text):
        onegrams = self.generate_ngrams(text, 1)
        bigrams = self.generate_ngrams(text, 2)
        trigrams = self.generate_ngrams(text, 3)

        onegram_fitness = 0
        if self.onegram_weight > 0:
            for onegram in onegrams:
                if onegram in self.onegram_frequency:
                    frequency = int(self.onegram_frequency[onegram])
                    onegram_fitness += math.log2(frequency)
        
        bigram_fitness = 0
        if self.bigram_weight > 0:
            for bigram in bigrams:
                if bigram in self.bigram_frequency:
                    frequency = int(self.bigram_frequency[bigram])
                    bigram_fitness += math.log2(frequency)
        
        trigram_fitness = 0
        if self.trigram_weight > 0:    
            for trigram in trigrams:
                if trigram in self.trigram_frequency:
                    frequency = int(self.trigram_frequency[trigram])
                    trigram_fitness += math.log2(frequency)
            
        fitness = (onegram_fitness * self.onegram_weight) + (bigram_fitness * self.bigram_weight) + (trigram_fitness * self.trigram_weight)
        return fitness

    # merge two parent to produce offspring
    def merge(self, one, two):
        offspring = [None] * 26
        count = 0

        while count < self.crossover_points_count:
            r = randint(0, len(one)-1)
            if offspring[r] == None:
                offspring[r] = one[r]
                count += 1
        for ch in two:
            if ch not in offspring:
                for i in range(len(offspring)):
                    if offspring[i] == None:
                        offspring[i] = ch
                        break
        return ''.join(offspring)

    # mutate key
    def mutate(self, key):
        a = randint(0, len(key)-1)
        b = randint(0, len(key)-1)

        key = list(key)
        key[a], key[b] = key[b], key[a]

        return ''.join(key)

    # initial population (random)
    def initial(self):
        pop = []

        for p in range(self.population_size):
            key = ''
            while len(key) < 26:
                r = randint(0, len(self.letters)-1)
                
                if self.letters[r] not in key:
                    key += self.letters[r]

            pop.append(key)

        return pop
    
    # evaluate all population fitness
    def evaluation(self, pop):
        fitness = []
        for key in pop:
            decrypted_txt = self.decrypt(key)
            key_fitness = self.calc_fitness(decrypted_txt)
            fitness.append(key_fitness)

        return fitness
    
    # take the top
    def elitism(self, pop, fitness):
        pop_fitness = {}

        for i in range(self.population_size):
            key = pop[i]
            val = fitness[i]

            pop_fitness[key] = val

        pop_fitness =  {k: v for k, v in sorted(pop_fitness.items(), key=lambda item: item[1])}
        sorted_population = list(pop_fitness.keys())

        elitist = sorted_population[-self.elitism_count:]

        return elitist

    # select the fittest individual and let their genes pass to the next generation.
    def tournament_selection(self, pop, fitness):
        pop_copy = pop.copy()
        selected_keys = []

        for a in range(2):
            tournament_pop = {}

            for t in range(self.tournament_size):
                r = randint(0, len(pop_copy)-1)
                key = pop_copy[r]
                key_fitness = fitness[r]

                tournament_pop[key] = key_fitness
                pop_copy.pop(r)

            sorted_tournament_pop = {k: v for k, v in sorted(tournament_pop.items(), key=lambda item: item[1], reverse=True)}
            tournament_keys = list(sorted_tournament_pop.keys())
            
            index = -1
            selected = False
            while not selected:
                index = randint(0, self.tournament_size-1)
                proba = self.tournament_proba[index]

                r = uniform(0, self.tournament_winner_proba)
                selected = (r < proba)
        
            selected_keys.append(tournament_keys[index])

        return selected_keys[0], selected_keys[1]

    # mutation the key (random)
    def mutation(self, pop, population_size):
        for i in range(population_size):
            r = uniform(0,1)

            if r < self.mutation_proba:
                key = pop[i]
                mutated = self.mutate(key)

                pop[i] = mutated
        return pop

    # reproduction to produce offspring, also called as crossover phase
    def repro(self, pop, fitness):
        crossover_pop = []

        while len(crossover_pop) < self.crossover_count:
            one, two = self.tournament_selection(pop, fitness)

            offspring_one = self.merge(one, two)
            offspring_two = self.merge(two, one)

            crossover_pop += [offspring_one, offspring_two]
        
        cross_pop = self.mutation(crossover_pop, self.crossover_count)
        return crossover_pop

    # the solve function
    def solve(self, ciphertext):
        file = open('/content/TheGreatGatsby.txt', 'r')

        text = file.read()
        file.close()
        self.ciphertext = ciphertext

        self.lettercase = [ch.islower() and ch.isalpha() for ch in self.ciphertext]
        self.ciphertext = self.ciphertext.upper()

        self.onegram_frequency = self.get_freq(text, 1)
        self.bigram_frequency = self.get_freq(text, 2)
        self.trigram_frequency = self.get_freq(text, 3)

        print(self.onegram_frequency)
        print(self.bigram_frequency)
        print(self.trigram_frequency)

        pop = self.initial()

        highest = 0
        
        for n in range(self.gen):
            fitness = self.evaluation(pop)
            elitist = self.elitism(pop, fitness)
            crossover_pop = self.repro(pop, fitness)

            pop = elitist + crossover_pop

            highest = max(fitness)
            avg = sum(fitness) / self.population_size

            idx = fitness.index(highest)
            key = pop[idx]
            decrypted_txt = self.decrypt(key)
            plaintxt = ''.join(decrypted_txt)        

            print('[Generation ' + str(n) + ']',)
            print('Average Fitness:', avg)
            print('Max Fitness:', highest)
            print('Key:', key)
            print('Decrypted Text:\n' + plaintxt + '\n')  

            if check(plaintxt): # if all of the words are english word then stop
                return plaintxt
            
        return plaintxt



In [ ]:
solver = GeneticSolver()
ciphertext = input()
plaintext = solver.solve(ciphertext)
print(plaintext)

MNSCXAMPRRPMCKXAPGSACMSGSAVKXACJSMNSVGKHSAACAPHKRRPBKGPMCKXUCMNMNSCDDSECPMSHKDDWXCMCSACXRKHPRAVPHSAMNSCXAMPRRPMCKXAHKXACAMSEKQMUKRPOSGAKQAKWXEPASGCSAKQHKDVKACMCKXAHGSPMSEQGKDRWRRPBCSAHKXMGCBWMSEBOQPMNSGAQGKDMNSCDDSECPMSHKDDWXCMOPXEPASMKQDKMCKXPHMCJPMSEAKWXEAMPMCKXAPWECSXHSAVGSASXHSVGKDVMADWRMCMWESAKQAMKGCSAMKWXQKREPWECKEKHWDSXMPGCSAHGSPMSECXHKXJSGAPMCKXUCMNQPMNSGAKXQSESGPRVGKBPMCKXCXAMPRRPMCKXAPGSPXKVSXHPRRMKMNSHKDDWXCMOMNGKWINPVPGMCHCVPMKGORKHPMCKXBPASEHKXMGCBWMKGOPWECKPWIDSXMSEGSPRCMOVRPMQKGDMNPMHPXBSSYVSGCSXHSEKXRCXSMNGKWINPDPVPXECXVNOACHPRAVPHSKGBOUPRLCXIMNSXSCINBKGNKKEAPAPXPWIDSXMSEPWECKPVVMNSDPVHGSPMSAPAKXCHGSVGSASXMPMCKXKQDSDKGCSARWRRPBCSAJCACKXAPXEESACGSAHKHGSPMCXIXSUHKRRSHMCJSAKHCPRDSDKGCSA
{'E': 25129, 'T': 18783, 'A': 17161, 'O': 15870, 'N': 14225, 'I': 14172, 'S': 12699, 'H': 12657, 'R': 11385, 'D': 9915, 'L': 8259, 'U': 5789, 'M': 5517, 'W': 5450, 'G': 4891, 'C': 4556, 'Y': 4545, 'F': 4199, 'B': 3188, 'P': 3067, 'K': 1966, 'V': 1900, 'J': 345, 'X': 287, 'Q': 160, 'Z': 145}